In [ ]:
import requests
from spec_constants import *
from visualization_constants import *
import numpy as np
import pandas as pd
import json
from numpy import genfromtxt

In [ ]:
# store login data in login.py
%run login.py

# login query as multiline formatted string
# this assumes that login and pwd are defined 
# above

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""


url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

# Getting Zone Data From Querying

In [ ]:
zone_ids = {scene:np.zeros((ROWS, COLUMNS)) for scene in SCENES}

# Read the zone IDs generated for the three areas
for camera_number in range(NUM_CAMERAS):
    zone_ids[CAMERA_TO_SCENE[camera_number]] = pd.read_csv(
        "./q1_zones/" + str(ROWS) + "x" + str(COLUMNS) + " " + CAMERA_TO_SCENE[camera_number] + ".csv", 
        header=None).values.astype(int)

# Contains the mean lingering time for each zone
data_arrays = [[[np.zeros((ROWS, COLUMNS, NUM_METRICS)) for i in range(NUM_CAMERAS)] 
                             for j in range(FRAMES)] for k in range(NUM_OBJECTS)]

In [ ]:
ROWS = 24
COLUMNS = 32

# For each zone in each of the three areas, find the metric values in that zone
for camera_number in range(NUM_CAMERAS):
    for zone in range(ZONE_NUM):
        # Determine the row and column that locates the zone so the its ID can be found.
        row = int(zone/COLUMNS)
        column = zone % COLUMNS
        # This query gets the object count and mean dwell time information of the zone for the pedestrian, 
        # bicycle, car, bus, and truck object classes. The metric are calculated each day for in the area specified
        # by camera_number.
        dwell_time_query = """
        query {
          zoneDwellTimeDistribution(
            zoneIds: """ + str(zone_ids[CAMERA_TO_SCENE[camera_number]][row, column]) + """,
            startTime:"2019-02-20T00:00:00",
            endTime:"2020-01-11T00:00:00",
            objClasses:["pedestrian", "bicycle", "car", "bus", "truck"],
            timezone:"America/New_York",
            interval:\"""" + str(TIME_INTERVAL) + """d\") {
            edges {
              node {
                mean
                count
              }
            }
          }
        }
        """
        # Send a request using the API to get the data.
        request = requests.post(url, json={'query': dwell_time_query}, headers = {'Authorization':token})
        # The coordinates the current zone are found before and the corresponding metric values are found below
        for data_point in range(FRAMES * NUM_OBJECTS):
            zone_data = json.loads(json.dumps(request.json()['data']['zoneDwellTimeDistribution']
                                        ['edges'][data_point]['node']))
            # Store the metric value into an array that represents the ROWS * COLUMNS zones. The information for 
            # each day is listed together in five entries corresponding to the object types in alphabetical order.
            data_arrays[data_point % NUM_OBJECTS][int(data_point/NUM_OBJECTS)][
                camera_number][row, column] = [zone_data['count'], zone_data['mean']]
        # Show the amount of processing that is completed.
        print(f"Finished processing {CAMERA_TO_SCENE[camera_number]} zone {zone + 1}/{ZONE_NUM}", 
              end="\r")

In [ ]:
import numpy
import pandas as pd

# Stores the data for each day of the zone heatmap concatenated together.
full_data = np.zeros((ROWS, COLUMNS))

# Replae all nan values in the data queried with 0.
data_arrays = np.nan_to_num(data_arrays)

# Write the metric values for each frame for each area into a csv file
for object_class in range(NUM_OBJECTS):
    for metric_index in range(NUM_METRICS):
        for frame in range(FRAMES):
            for camera_number in range(NUM_CAMERAS):
                # The data is a ROW by COLUMN array that stores the data for a specific object class, day, area, and
                # metric. The object class needs to be put into a certain order that is different from the querying 
                # order for the heatmap dropdown so the index for the current object type in the queried data needs 
                # to be found.
                data = data_arrays[OBJECT_CLASSES_QUERY.index(OBJECT_CLASSES[object_class])][
                    frame][camera_number][:, :, metric_index]
                # Add the ROW by COLUMN array to the array storing all the data.
                full_data = np.concatenate((full_data, data))
        # Show the amount of processing that is completed.        
        print(f"Finished processing object class {object_class} metric {metric_index}")

In [ ]:
# Write the data into a csv file so the querying needs to be done only once.
np.savetxt("q1_full_data.csv", full_data[ROWS:, :], fmt='%.4e', delimiter=",")

# Generating Absorption Index Data

Since the absorption index is computed based on queried data, placing the absorption index values into an array needs to be done separately.

In [ ]:
import numpy as np

# Stores all of the absorption index data for each object type, day, and area.
absorption_data = np.zeros((ROWS, COLUMNS))

# For each moment specified by an object class, day, and area, find the absorption index for each zone.
for object_class in range(NUM_OBJECTS):
    for frame in range(FRAMES):
        for camera_number in range(NUM_CAMERAS):
            # The object class needs to be put into a certain order that is different from the querying order
            # for the heatmap dropdown so the index for the current object type in the queried data needs to be found.
            data = data_arrays[OBJECT_CLASSES_QUERY.index(OBJECT_CLASSES[object_class])][
                frame][camera_number][:, :, 0]
            # Stores the absorption index data for the zones in the frame specified by the object type, day, and area.
            frame_absorption_data = np.zeros((ROWS, COLUMNS))
            # Only zones that are not adjacent to the perimeter of the area are considered.
            for i in range(1, ROWS - 1):
                for j in range(1, COLUMNS - 1):
                    # For each zone, find the number of objects from the non-diagonally neighbouring four zones
                    neighbour_zone_total = data[i - 1, j] + data[i, j - 1] + data[i, j + 1] + data[i + 1, j]
                    # If the denominator in the formula is negative, then the value is set to 0.
                    if neighbour_zone_total != 0:
                        frame_absorption_data[i, j] = float(data[i, j]/neighbour_zone_total)
            # Add the frame's data into the array holding the data for all frames.
            absorption_data = np.concatenate((absorption_data, frame_absorption_data))
    # Show the amount of processing that is completed.  
    print(f"Finished processing object class {object_class}")

In [ ]:
# Write the data into a csv file so the computation needs to be done only once.
np.savetxt("absorption_index_data.csv", absorption_data[ROWS:, :], fmt='%.4e', delimiter=",")

# Generating Cumulative Object Count Data

The cumulative object count for a day and a zone is the total number of objects passing through the zone from the time the sensors were on to that day. This data will be recursively found using the pre-queried data.

In [ ]:
import numpy as np

# Stores all of the cumulative object count data.
cumulative_count_data = np.zeros((ROWS, COLUMNS))

# For each moment specified by an object class, day, and area, find the cumulative object count for all zones.
for object_class in OBJECT_CLASSES:
    # Stores the cumulative count arrays for each area. The arrays are updated after each frame.
    current_frame_arrays = np.zeros((ROWS, COLUMNS, NUM_CAMERAS))
    for frame in range(FRAMES):
        for camera_number in range(NUM_CAMERAS):
            # The object count data for this frame.
            frame_data = HEATMAP_ARRAYS[object_class]['Object Count'][frame][camera_number]
            # Add the object count data to the current cumulative object count.
            current_frame_arrays[:, :, camera_number] += frame_data
            # Append the new cumulative object count to the array holding all data.
            cumulative_count_data = np.concatenate((cumulative_count_data, current_frame_arrays[:, :, camera_number]))
    # Show the amount of processing that is completed.
    print(f"Finished processing object class {object_class}")

In [ ]:
# Write the data into a csv file so the computation needs to be done only once.
np.savetxt("cumulative_count_data.csv", cumulative_count_data[ROWS:, :], fmt='%.4e', delimiter=",")

# Getting Area Data

We previously divided the areas into zones and found data for each of the zones. We will now get data for the entire area.

In [ ]:
# Stores the data for each day, area, metric, and object type.
area_data = {object_class:[[np.zeros((1, NUM_METRICS)) for i in range(NUM_CAMERAS)] 
                             for j in range(FRAMES)] for object_class in OBJECT_CLASSES}

# For each of the three areas, get dwell time and traffic statistics for each day.
for camera_number in range(NUM_CAMERAS):
    # This query gets the object count and mean dwell time information of the zone for the pedestrian, 
    # bicycle, car, bus, and truck object classes. The metric are calculated each day for in the area specified
    # by camera_number.
    area_stats_query = """
    query {
      zoneDwellTimeDistribution(
        zoneIds: """ + str(AREA_IDS[camera_number]) + """,
        startTime:"2019-02-20T00:00:00",
        endTime:"2020-01-11T00:00:00",
        objClasses:["pedestrian", "bicycle", "car", "bus", "truck"],
        timezone:"America/New_York",
        interval:\"""" + str(TIME_INTERVAL) + """d\") {
        edges {
          node {
            pct25
            pct50
            pct75
            mean
            count
            objClass
          }
        }
      }
    }
    """  
    # Send a request using the API to get the data.
    request = requests.post(url, json={'query': area_stats_query}, headers = {'Authorization':token})
    # The coordinates the current zone are found before and the corresponding metric values are found below.
    for data_point in range(FRAMES * NUM_OBJECTS):
        # Get the data for the current day and object type.
        zone_data = json.loads(json.dumps(request.json()['data']['zoneDwellTimeDistribution']
                                    ['edges'][data_point]['node']))
        # Determine the object type the data is for.
        zone_object = zone_data['objClass']
        # Contains values of the metrics for the area
        zone_values = list(zone_data.values())
        # The data for all the metrics except the object type is stored.
        zone_values.remove(zone_object)
        # Place the data for each of the metrics in place in the array corresponding to the object type, day, and
        # area.
        area_data[zone_object][int(data_point/NUM_OBJECTS)][
            camera_number] = np.array([0 if value == None else value for value in zone_values])
    # Show the amount of processing that is completed. 
    print(f"Finished processing {CAMERA_TO_SCENE[camera_number]}", 
          end="\r")

In [ ]:
import numpy as np

# Stores all of the area data in an array to be written to a csv file.
area_csv_data = np.zeros((1, FRAMES))

# For the data for each object type, metric, and area, and day.
for object_class in OBJECT_CLASSES:
    for metric in range(NUM_METRICS):
        for camera_number in range(NUM_CAMERAS):
            # The data for each day is stored in a list.
            data = np.array([[area_data[object_class][frame][camera_number][metric] for frame in range(FRAMES)]])
            # Add the list to the array containing all the data.
            area_csv_data = np.concatenate((area_csv_data, data))

In [ ]:
# Write the data into a csv file so the processing needs to be done only once.
np.savetxt("area_csv_data.csv", area_csv_data[1:], delimiter=",")